In [2]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from datetime import datetime, timedelta, timezone
pd.set_option('display.float_format', lambda x: '%.5f' % x) #soppressione della notazione scientifica
from math import radians, cos, sin, asin, sqrt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from google.colab import drive

from keras import *
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from keras.backend import abs, sum
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import os 
import statistics
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train = pd.read_csv(r"/content/drive/MyDrive/AML project/train.csv", parse_dates=['pickup_datetime'])

In [5]:
display(train.head())

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.98215,40.76794,-73.96463,40.76560,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.98042,40.73856,-73.99948,40.73115,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.97903,40.76394,-74.00533,40.71009,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.01004,40.71997,-74.01227,40.70672,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.97305,40.79321,-73.97292,40.78252,N,435


In [6]:
train.shape

(1458644, 11)

In [7]:
train = train.drop(columns = ['dropoff_datetime'])

Si inizia cercando eventuali valori mancanti all'interno del train e del test

In [8]:
# ricerca valori mancanti
print("valori mancanti trainig set")
print(train.isnull().sum())

valori mancanti trainig set
id                    0
vendor_id             0
pickup_datetime       0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64


Accerttata l'assenza di valori mancanti, si passa a calcolare la distanza tra il punto di prelievo ed il punto di destinazione, tramite la distanza di Harversine. La distanza di Harvesine, calcola la *great-circle distance* tra due punti su una sfera, date la latitudine e la longitudine  dei due punti. L'ipotesi si sfericità è semplificatrice, ma sulle piccole distanze (come quelle di questo dataset) tale ipotesi non ha un impatto rilevante sul calcolo. La distanza di Harvesine è definita dalla seguente formula:

$$\displaystyle d = 2r\arcsin\left( \sqrt{\sin^{2}\left( \frac{\varphi_{2} - \varphi_1}{2} \right) 
+\cos\left( \varphi_1 \right)\cos\left( \varphi_2 \right) \sin^{2}\left( \frac{\lambda_2-\lambda_1}{2} \right)
} \right)$$

dove $\varphi_1$  e $\varphi_2$ sono la latitudine del punto 1 del punto 2 in radianti, mentre $\lambda_1$ e $\lambda_2$ sono la longitudine del punto 1 e del punto 2. Invece $r$ è la raggio della sfera (in questo caso il raggio della terra). Ovviamente, questo tipo di distanza non considera la mappa della città, tuttavia l'unico modo per ottenere una distanza in grado di considerare il vero scenario del traffico è necessario utilizzare le API di Google maps, che tuttavia sono a pagamento.

In [9]:
#funzione per la distanza di Harvesine espressa in kilometri

def haversine(x):
  # conversione in radianti

    lon1, lon2, lat1, lat2 = x['pickup_longitude'], x['pickup_latitude'], x['dropoff_longitude'], x['dropoff_latitude']
    lon1, lon2, lat1, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
# haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6372.797 # Raggio medio della Terra in chilometri.
    return round(c * r, 4)

In [10]:
train['distance'] = train[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']].apply(haversine, axis = 1)

In [11]:
def get_direction(lng1, lat1, lng2, lat2):
    # theta
    AVG_EARTH_RADIUS = 6371  #  km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

In [12]:
train['direction'] = pd.DataFrame(get_direction(train['pickup_longitude'].values,  train['pickup_latitude'].values, 
                                              train['dropoff_longitude'].values, train['dropoff_latitude'].values))

Calcolata la distanza, passiamo ad estrarre 7 variabili dummies, una per giorno della settimana. 

In [13]:
#per ogni datetime si estrae il giorno della settimana in cui è avvenuto il viaggio

train['name_day'] = train['pickup_datetime'].apply(lambda x: x.date().strftime('%A'))

In [14]:
# si procede ad ottenere le dummies

train = pd.get_dummies(train, columns = ['name_day'])

Si procede a rendere binaria, la variabile *store_and_fwd_flag*

In [15]:
train['store_and_fwd_flag'] = train['store_and_fwd_flag'].apply(lambda x: 1 if (x == "N") else 0)

In [16]:
display(train.head())

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,direction,name_day_Friday,name_day_Monday,name_day_Saturday,name_day_Sunday,name_day_Thursday,name_day_Tuesday,name_day_Wednesday
0,id2875421,2,2016-03-14 17:24:55,1,-73.98215,40.76794,-73.96463,40.76560,1,455,1.49890,99.97020,0,1,0,0,0,0,0
1,id2377394,1,2016-06-12 00:43:35,1,-73.98042,40.73856,-73.99948,40.73115,1,663,1.80600,-117.15377,0,0,0,1,0,0,0
2,id3858529,2,2016-01-19 11:35:24,1,-73.97903,40.76394,-74.00533,40.71009,1,2124,6.38690,-159.68017,0,0,0,0,0,1,0
3,id3504673,2,2016-04-06 19:32:31,1,-74.01004,40.71997,-74.01227,40.70672,1,429,1.48590,-172.73770,0,0,0,0,0,0,1
4,id2181028,2,2016-03-26 13:30:55,1,-73.97305,40.79321,-73.97292,40.78252,1,435,1.18890,179.47358,0,0,1,0,0,0,0


In [17]:
train['pickup_month'] = [x.month for x in train['pickup_datetime']]

In [18]:
# creo le dummies dei mesi partendo dalla colonna pickup_month
month_mapping = {
    
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

train = train.join(pd.get_dummies(train['pickup_month'].map(month_mapping)))

In [19]:
# creo un vettore delle festività di NewYork
cal = calendar()
holidays = cal.holidays(start = (train['pickup_datetime'].min()) - timedelta(days=1), end=train['pickup_datetime'].max())
holidays

DatetimeIndex(['2016-01-01', '2016-01-18', '2016-02-15', '2016-05-30'], dtype='datetime64[ns]', freq=None)

In [20]:
# funzione che verifica che il giorno in oggetto sia un business day o un giorno festivo
def is_business_day(date):
    
    if date in holidays or bool(len(pd.bdate_range(date, date))) is False:
        return 0
    else: 
        return 1 

In [21]:
# creo la colonna 0 1. Se 0 vuol dire che è un giono festivo, se 1 è un giorno di lavorio
train['business_day'] = [is_business_day(x) for x in train['pickup_datetime']]

In [22]:
# creo una colonna con solo l'ora
train['time_hour'] = pd.to_datetime(train['pickup_datetime']).dt.hour

In [23]:
train = pd.get_dummies(train, columns = ['time_hour'])

In [24]:
#eliminazione colonne inutili

drop_columns = ['id', 'vendor_id', 'pickup_longitude', 'pickup_latitude',
                'dropoff_longitude', 'dropoff_latitude', 'pickup_month']

train = train.drop(columns = drop_columns)

## Integrazione dati meteo

In [25]:
dati_meteo = pd.read_csv(r"/content/drive/MyDrive/AML project/weather_data_nyc_centralpark_2016.csv",
                  parse_dates=['date'])

In [26]:
dati_meteo['snow fall'] = dati_meteo['snow fall'].str.replace('T', '0.00')
dati_meteo['precipitation'] = dati_meteo['precipitation'].str.replace('T', '0.00')
dati_meteo['snow depth'] = dati_meteo['snow depth'].str.replace('T', '0.00')

In [27]:
dati_meteo['snow fall'] = dati_meteo['snow fall'].apply(lambda x: float(x))
dati_meteo['snow depth'] = dati_meteo['snow depth'].apply(lambda x: float(x))
dati_meteo['precipitation'] = dati_meteo['precipitation'].apply(lambda x: float(x))

In [28]:
train['date'] = train['pickup_datetime'].dt.date

In [29]:
dati_meteo.date = dati_meteo.date.astype(str)
train.date = train.date.astype(str)

In [30]:
train = pd.merge(train, dati_meteo, on = 'date')

In [31]:
train = train.drop(columns = ['date', 'pickup_datetime'])

## Analisi esplorativa

In [32]:
display("statistiche descrittive train", train.describe())

'statistiche descrittive train'

,passenger_count,store_and_fwd_flag,trip_duration,distance,direction,name_day_Friday,name_day_Monday,name_day_Saturday,name_day_Sunday,name_day_Thursday,name_day_Tuesday,name_day_Wednesday,April,February,January,June,March,May,business_day,time_hour_0,time_hour_1,time_hour_2,time_hour_3,time_hour_4,time_hour_5,time_hour_6,time_hour_7,time_hour_8,time_hour_9,time_hour_10,time_hour_11,time_hour_12,time_hour_13,time_hour_14,time_hour_15,time_hour_16,time_hour_17,time_hour_18,time_hour_19,time_hour_20,time_hour_21,time_hour_22,time_hour_23,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
count,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000,1458644.00000
mean,1.66453,0.99448,959.49227,3.44183,-15.66892,0.15325,0.12849,0.15142,0.13394,0.14985,0.13900,0.14406,0.17252,0.16337,0.15748,0.16064,0.17564,0.17035,0.71464,0.03651,0.02644,0.01918,0.01432,0.01083,0.01028,0.02279,0.03812,0.04597,0.04639,0.04486,0.04694,0.04927,0.04900,0.05093,0.04923,0.04409,0.05243,0.06211,0.06191,0.05764,0.05771,0.05518,0.04784,61.76372,46.73164,54.24768,0.08850,0.05858,0.40647
std,1.31424,0.07406,5237.43172,4.29775,104.49149,0.36023,0.33463,0.35846,0.34058,0.35692,0.34594,0.35115,0.37783,0.36970,0.36425,0.36720,0.38051,0.37594,0.45158,0.18754,0.16045,0.13715,0.11883,0.10349,0.10089,0.14925,0.19148,0.20942,0.21032,0.20700,0.21152,0.21644,0.21587,0.21986,0.21635,0.20530,0.22290,0.24136,0.24100,0.23306,0.23320,0.22834,0.21343,16.97841,15.71658,16.14521,0.23367,0.93980,2.24661
min,0.00000,0.00000,1.00000,0.00000,-179.99270,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,15.00000,-1.00000,7.00000,0.00000,0.00000,0.00000
25%,1.00000,1.00000,397.00000,1.23220,-125.27359,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,49.00000,36.00000,42.50000,0.00000,0.00000,0.00000
50%,1.00000,1.00000,662.00000,2.09430,8.20000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,61.00000,46.00000,53.50000,0.00000,0.00000,0.00000
75%,2.00000,1.00000,1075.00000,3.87640,53.40495,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,76.00000,61.00000,69.50000,0.04000,0.00000,0.00000
max,9.00000,1.00000,3526282.00000,1241.25870,180.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000

In [33]:
display("statistiche descrittive test", test.describe())

NameError: ignored

In [34]:
#elimare righe con più di 100 km
#eliminarrighe con meno di 100 metri
#eliminare righe durata meno di un secondo
#eliminare righe durata più di 6 ore di viaggio
#eliminare i records con un numero di passeggeri uguale a zero (sono solo 51)

In [35]:
#eliminare i records con un numero di passeggeri uguale a zero (sono solo 51)

train = train[train['trip_duration'] >= 60]
train = train[train['trip_duration'] < 21600]
train = train[train['distance'] >= 0.1]
train = train[train['distance'] <= 100]
train = train[train['passenger_count'] != 0]

In [36]:
train.to_csv(r"C:\Users\39331\Desktop\AML project\train_manipulated_integrated.csv", index = False)